In [ ]:
from collections import Iterable
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import pyvista as pv
import vtk

In [ ]:
def to_unstructured(fname, node_face, node_x, node_y, radius=None, start_index=True, panel=None, data=None, step=None):
    if radius is None:
        radius = 1.0
        
    with nc.Dataset(fname) as ds:
        node_face = ds.variables[node_face][:].data
        node_x = ds.variables[node_x][:].data
        node_y = ds.variables[node_y][:].data
        if data is not None and step is not None:
            data = ds.variables[data][:]
            mask = data.mask
            data = data.data
            data[mask] = np.nan
        
    # account for start_index = 1
    if start_index:
        node_x = np.concatenate(([0], node_x))
        node_y = np.concatenate(([0], node_y))
    
    # convert lat/lon to cartesian coordinates
    node_face_x = node_x[node_face]
    node_face_y = 90.0 - node_y[node_face]
    
    node_face_x_rad = np.radians(node_face_x)
    node_face_y_rad = np.radians(node_face_y)
    
    x = radius * np.sin(node_face_y_rad) * np.cos(node_face_x_rad)
    y = radius * np.sin(node_face_y_rad) * np.sin(node_face_x_rad)
    z = radius * np.cos(node_face_y_rad)
    
    # set the VTK cell type and number of vertices
    vtk_cell_type = vtk.VTK_QUAD
    N_nodes = 4
    N_panels = 6
    
    # create unstructured grid
    points = np.vstack((np.ravel(x), np.ravel(y), np.ravel(z))).T
    
    if panel is not None:
        PN = points.shape[0] // N_panels
        
        if isinstance(panel, Iterable):
            panel_points = []
            data_points = []
            
            for p in sorted(panel):
                panel_points.append(points[p*PN:(p+1)*PN])
                if data is not None:
                    data_points.append(data[step][p*(PN//N_nodes):(p+1)*(PN//N_nodes)])
                
            points = np.concatenate(tuple(panel_points))
            if data is not None:
                data_points = np.concatenate(tuple(data_points))
        else:
            points = points[panel*PN:(panel+1)*PN]
            if data is not None:
                data_points = data[step][panel*(PN//N_nodes):(panel+1)*(PN//N_nodes)]
    else:
        if data is not None:
            data_points = data[step]
            
    N_points = points.shape[0]
    N_faces = N_points // N_nodes
    N_faces_per_panel = N_faces // N_panels
    print(N_points, N_faces, N_faces_per_panel, N_nodes)
    cell_type = np.broadcast_to(np.array([vtk.VTK_QUAD], np.uint8), (N_faces,))
    cells = np.ravel(np.hstack((np.broadcast_to(np.array([N_nodes], np.int8), (N_faces, 1)),
                                np.arange(0, N_points).reshape((-1, N_nodes)))))
    offset = np.arange(0, cells.shape[0], N_nodes + 1)
    
    ugrid = pv.UnstructuredGrid(offset, cells, cell_type, points)
    
    if data is not None:
        ugrid.cell_arrays["faces"] = data_points
    
    return ugrid

In [ ]:
fname = "/home/bill/pyvista/data/test/real/qrclim.sst.ugrid.nc"
node_face = "dynamics_face_nodes"
node_x = "dynamics_node_x"
node_y = "dynamics_node_y"
data = "surface_temperature"
step = 0
panel = None

In [ ]:
for s in range(12):
    ugrid = to_unstructured(fname, node_face, node_x, node_y, data=data, step=s, panel=None)
    mesh_fname = f"ugrid_sst_t{s}.vtk"
    ugrid.save(mesh_fname)
    for p in range(6):
        ugrid = to_unstructured(fname, node_face, node_x, node_y, data=data, step=s, panel=p)
        mesh_fname = f"ugrid_sst_t{s}_panel{p}.vtk"
        ugrid.save(mesh_fname)

In [ ]:
!ls -l ugrid*.vtk

In [ ]:
ugrid = to_unstructured(fname, node_face, node_x, node_y, data=data, step=step, panel=panel)

In [ ]:
ugrid

In [ ]:
p = pv.Plotter()
p.add_mesh(ugrid)
#p.add_points(ugrid.points, color="green")
p.show()

In [ ]:
mesh = pv.read(mesh_fname)

In [ ]:
mesh

In [ ]:
p = pv.PlotterITK()
p.add_mesh(mesh)
#p.add_points(mesh.points, color="red")
p.show()